In [29]:
# My OpenAI Key
import os
# read key from file
with open('..\..\.env', 'r') as fp:
    os.environ['OPENAI_API_KEY'] = fp.read().strip()

In [30]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Using Knowledge Graph

#### Building the Knowledge Graph

In [31]:
from gpt_index import SimpleDirectoryReader, LLMPredictor
from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI
from IPython.display import Markdown, display

In [32]:
documents = SimpleDirectoryReader('../dk_b/data').load_data()

In [33]:
# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-002"))

In [34]:
# NOTE: can take a while! 
index = GPTKnowledgeGraphIndex(
    documents, 
    chunk_size_limit=512, 
    max_triplets_per_chunk=2,
    llm_predictor=llm_predictor
)

INFO:root:> [build_index_from_documents] Total LLM token usage: 191147 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens


In [35]:
index.save_to_disk('index_dk.json')

In [36]:
# try loading
new_index = GPTKnowledgeGraphIndex.load_from_disk('index_dk.json', llm_predictor=llm_predictor)

#### Querying the Knowledge Graph

In [37]:
response = new_index.query(
    "Tell me more about protoyping", 
    include_text=False, 
    response_mode="tree_summarize"
)

INFO:root:> Starting query: Tell me more about protoyping
INFO:root:> Query keywords: ['definition', 'stages', 'advantages', 'Prototyping', 'disadvantages', 'process']
INFO:root:> Extracted relationships: The following are knowledge triplets in the form of (subset, predicate, object):
('Prototyping', 'as an opportunity', 'to transform the future')
('Prototyping', 'driven by', 'curiosity')
('Prototyping', 'drives', 'curiosity')
('Prototyping', 'connects', 'knowledge and living')
('Prototyping', 'materializes', 'futures')
('Prototyping', 'without teleology becomes', 'plea for agency and freedom')
('Prototyping', 'transforms', 'society')
('Prototyping', 'is', 'model for governance')
('Prototyping', 'revives values of', 'artisanal knowledge')
('Prototyping', 'becomes', 'receptacle')
('Prototyping', 'supports', 'existing structures')
('Prototyping', 'provokes us to imagine', 'a state based on something other than religion')
('Prototyping', 'serves', 'many functions')
('Prototyping', 'an exp

In [38]:
display(Markdown(f"<b>{response}</b>"))

<b>
Prototyping is a form of "doing" philosophy that allows individuals to monopolize the creation of tools and community. It is an expression of Kairos, which opposes Chronos.</b>

In [39]:
response = new_index.query(
    "tell me more about sandboxes", 
    include_text=True, 
    response_mode="tree_summarize"
)

INFO:root:> Starting query: tell me more about sandboxes
INFO:root:> Query keywords: ['more', 'tell', 'sandboxes']
INFO:root:> Querying with idx: 0eb13328-472a-4195-ac53-12f53aae8e71: Inspired by the liminal environments of Chapter 3 and the forgotten history o...
INFO:root:> Querying with idx: 8a7b4c7c-33f8-4465-9c0a-eedc8e086904: al., 2021), sandboxes allow tactical and hybrid decisions on values and code....
INFO:root:> Querying with idx: da3c994e-a4ff-4aa7-8eee-6212ecf4a567: resignation upon innovation in the name of the
public good. Instead of replac...
INFO:root:> Querying with idx: 2aa0f27a-0fb7-4e7f-ad7b-e00fefd64b8c: ethnography and genealogy provides examples for the proposal
for sandboxes as...
INFO:root:> Querying with idx: c98657e6-f85e-4b47-b992-2351d66a7a60: any regulatory
force or domain (market, culture, technology, or law).
4.2 Exp...
INFO:root:> Querying with idx: 7d8aaf5c-af48-4866-9f59-36c58bb176a7: governance.
     The fantasies of algorithmic rule and automation 

In [40]:
display(Markdown(f"<b>{response}</b>"))

<b>
Sandboxes are temporary and experimental environments that allow stakeholders to negotiate and define common goals for the future. They prevent the monopolization of the future by one set of interests and values, and emphasize the experience of Kairos, agency, and capacity for transformation.

Sandboxes are environments where governance meets prototyping. They are liminal environments that support explorations of the material, social, and technological conditions of governance over prototypes. Sandboxes can be used to test out new technologies and ideas in a controlled environment, which can help prevent arbitrary excesses and support temporary and even plural futures.</b>

#### Visualizing the Graph

In [41]:
## create graph
from pyvis.network import Network

g = new_index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("dk_graph.html")